In [2]:
import requests
from bs4 import BeautifulSoup
import time
import urllib.request
from selenium.webdriver import Chrome
import re
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
import datetime as dt
import pandas as pd

In [ ]:
# 전체

In [38]:
delay = 3
path = "C:\dev\driver/chromedriver.exe"
headers = {
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/69.0.3497.100 Safari/537.36'}
browser = Chrome(path)
browser.implicitly_wait(delay)

In [39]:
comment_data = pd.DataFrame({'youtube_id': [],
                             'comment': [],
                             'like_num': []})
browser = Chrome(path)  # chrome_options=options
browser.implicitly_wait(delay)

In [40]:
start_url = 'https://www.youtube.com'
browser.get(start_url)
# browser.maximize_window()

In [41]:
browser.find_elements_by_xpath(
    '//*[@id="search-form"]/div/div[1]')[0].click()  # 검색창영역클릭

browser.find_elements_by_xpath(
    '//*[@class="gstl_50 sbib_a"]/div[2]/input')[0].click()
browser.find_elements_by_xpath(
    '//*[@class="gstl_50 sbib_a"]/div[2]/input')[0].send_keys('타다')
browser.find_elements_by_xpath(
    '//*[@class="gstl_50 sbib_a"]/div[2]/input')[0].send_keys(Keys.RETURN)  # 엔터

In [42]:
browser.find_elements_by_xpath(
    '//*[@id="container"]/ytd-toggle-button-renderer')[0].click()
browser.find_elements_by_xpath(
    '//*[@id="collapse-content"]/ytd-search-filter-group-renderer[1]/ytd-search-filter-renderer[3]/a')[0].click()
# browser.find_elements_by_xpath('//*[@id="collapse-content"]/ytd-search-filter-group-renderer[1]/ytd-search-filter-renderer[3]/a').send_keys(Keys.RETURN)

In [43]:
body = browser.find_element_by_tag_name('body')  # 스크롤하기 위해 소스 추출

num_of_pagedowns = 10
# 5번 밑으로 내리는 것
while num_of_pagedowns:
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(2)
    num_of_pagedowns -= 1

In [44]:
html0 = browser.page_source
html = BeautifulSoup(html0, 'html.parser')

In [45]:
# <ytd-thumbnail class="style-scope ytd-video-renderer" use-hovered-property="">
# 검색에서 주소 가져오기

# 작동코드 건들지 말 것
video_list0 = html.find('div', {'id': 'contents'})
video_list1 = html.find_all(
    'a', {'class': 'yt-simple-endpoint style-scope ytd-video-renderer'})
video_list1
#video_list1 = html.find_all('div', {'class': 'style-scope ytd-video-renderer'})
#video_list2 = video_list0.find_all('div', {'id': 'thumbnail'})
#video_list2 = video_list1.find_all('div', {'class': 'style-scope ytd-video-renderer'})
# video_list0[1].find('a')['href']
#title1 = video_list0[1].find('a',{'id':'video-title'}).text
# title1
# video_list1[0]['href']
# //*[@id="metadata-line"]/span[1]/text()
video_list2 = video_list0.find_all(
    'span', {'class': 'style-scope ytd-video-meta-block'})
video_list2[0]

video_list3 = html.find_all(
    'div', {'class': 'text-wrapper style-scope ytd-video-renderer'})
video_list3[0].find('div', {'id': 'metadata-line'})

<div class="style-scope ytd-video-meta-block" id="metadata-line">
<span class="style-scope ytd-video-meta-block">조회수 6.7천회</span>
<span class="style-scope ytd-video-meta-block">6일 전</span>
<dom-repeat class="style-scope ytd-video-meta-block" strip-whitespace=""><template is="dom-repeat"></template></dom-repeat>
</div>

In [46]:
# 영상 주소 가져오기. 댓글, 좋아요 가져올 때 필요함
base_url = 'https://www.youtube.com'
test_url = []
for i in range(1, 30):
    url2 = base_url+video_list1[i]['href']
    test_url.append(url2)

In [47]:
start_url = test_url[0]
browser.get(start_url)
body = browser.find_element_by_tag_name('body')

In [48]:
comment_data = pd.DataFrame({'title': [],
                             'youtube_id': [],
                             'comment': [],
                             'like_num': []
                             })
browser = Chrome(path)  # chrome_options=options
browser.implicitly_wait(delay)

In [49]:
for i in range(1, 3):
    start_url = test_url[i]
    browser.get(start_url)
    body = browser.find_element_by_tag_name('body')

    time.sleep(2)

    num_page_down = 1
    while num_page_down:
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(1.5)
        num_page_down -= 1

    browser.find_element_by_xpath(
        '//paper-button[@class="dropdown-trigger style-scope yt-dropdown-menu"]').click()
    browser.find_element_by_xpath(
        '//paper-listbox[@class="dropdown-content style-scope yt-dropdown-menu"]/a[1]').click()

    num_page_down = 5
    while num_page_down:
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(1.5)
        num_page_down -= 1

    html_s0 = browser.page_source
    html_s = BeautifulSoup(html_s0, 'html.parser')

    comment0 = html_s.find_all(
        'ytd-comment-renderer', {'class': 'style-scope ytd-comment-thread-renderer'})
    title0 = html_s.find_all('ytd-video-primary-info-renderer')
    title = title0[0].find('h1').text.split('/')[0]
    for i in range(len(comment0)):

        # 댓글
        comment = comment0[i].find(
            'yt-formatted-string', {'id': 'content-text', 'class': 'style-scope ytd-comment-renderer'}).text

        try:
            aa = comment0[i].find('span', {'id': 'vote-count-left'}).text
        # 정규표현식으로 숫자만 추출하는 것은 정규표현식에 대한 공부를 더 한 뒤 해결
        # re.findall('[0-9]',aa)
        # "".join(re.findall('[0-9]',aa)) -> 리스트 내부의 문자열의 합
            like_num = "".join(re.findall('[0-9]', aa)) + "개"
        except:
            like_num = 0

        bb = comment0[i].find('a', {'id': 'author-text'}).find('span').text
        youtube_id = "".join(re.findall('[가-힣0-9a-zA-Z]', bb))

        insert_data = pd.DataFrame({'title': [title],
                                    'youtube_id': [youtube_id],
                                    'comment': [comment],
                                    'like_num': [like_num]
                                    })

        comment_data = comment_data.append(insert_data)

    comment_data.index = range(len(comment_data))

In [50]:
comment_data

,title,youtube_id,comment,like_num
0,'타다 베이직' 4월 11일 중단…1만명 넘는 드라이버 어쩌나,MarsGreener,사용자를 위해 택시가 있는건가요 아니면 택시를 위해 이 나라가 존재하는 건가요?? ...,0개
1,'타다 베이직' 4월 11일 중단…1만명 넘는 드라이버 어쩌나,qweqw,타다 편리하고 좋던데\n무조건 없에는건 답이 아니다,0개
2,'타다 베이직' 4월 11일 중단…1만명 넘는 드라이버 어쩌나,MyrtletreeChae,해외 선진국엔 이미 수년전부터 활성화되어있는데.... 호주엔 우버택시란 이름으로 ....,1개
3,'타다 베이직' 4월 11일 중단…1만명 넘는 드라이버 어쩌나,박혜근,타다 금지 시켜서 좋아 했지? 이젠 너희들이 쫓겨 나게 될거야,0개
4,'타다 베이직' 4월 11일 중단…1만명 넘는 드라이버 어쩌나,마을,타다기사 여러분 !! 지금 당장 택시회사로 오십시요! 당일 취업됩니다.,0개
5,'타다 베이직' 4월 11일 중단…1만명 넘는 드라이버 어쩌나,김건이,4월11일 부로 타다 베이직 그만두면\n그동안 어떤 대우를 받았는지 타다 운전자들 ...,0개
6,'타다 베이직' 4월 11일 중단…1만명 넘는 드라이버 어쩌나,하지마,타다가 1500대 람서헌데 기사가 1만명? 엉터리수치구나\n그런 엉터리 수치로 사...,0개
7,'타다 베이직' 4월 11일 중단…1만명 넘는 드라이버 어쩌나,김GT,사요나라..,0개
8,'타다 베이직' 4월 11일 중단…1만명 넘는 드라이버 어쩌나,박정선,이게 말이되나?,0개
9,'타다 베이직' 4월 11일 중단…1만명 넘는 드라이버 어쩌나,왜구참수,타다... 다른말로 하면 '나라시'아닌가?,0개
